# 1. Imports & Environment Setup

- **Core Imports:**
  - `os`, `warnings`, `datetime`, `uuid`, `string`, `random`, `hashlib`, `traceback`.

- **LangChain/LangGraph Imports:**
  - `ChatOpenAI`, `StreamingStdOutCallbackHandler`, `TavilySearch`, `TavilySearchResults`, agent/graph/integration tools, message types.

- **Notebook & Display:**
  - `clear_output`, `Markdown` from `IPython.display`.

- **Environment:**
  - Suppresses TensorFlow warnings.
  - Loads environment variables (e.g., API keys) using `python-dotenv`.


In [1]:
import os
import warnings
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_tavily import TavilySearch
from langchain_community.tools import TavilySearchResults
from langchain.agents import create_tool_calling_agent, AgentExecutor
from typing import TypedDict, Dict, List
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage
from datetime import datetime
from IPython.display import clear_output, Markdown
import traceback
import uuid
import string
import random
import hashlib

# 2. Key Utility Functions

get_api_key(key_name="OPENROUTER_API_KEY"):
Retrieves an API key from environment variables, raises an error if not found.

initialize_llm(...):
Initializes the language model (LLM) with streaming and callback support.

save_file(data, filename, uniqueness_method="uuid"):
Saves output (e.g., generated plans) to a Markdown file with various unique-naming strategies.

show_md_file(file_path):
Displays a Markdown file’s content within the notebook.

In [2]:
# Suppress TensorFlow and CUDA warnings for a cleaner notebook output
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
warnings.filterwarnings("ignore")

# Load all environment variables (such as API keys) from .env file
load_dotenv()

def get_api_key(key_name="OPENROUTER_API_KEY"):
    """
    Retrieve an API key from environment variables.

    Args:
        key_name (str): Name of the environment variable.

    Returns:
        str: Retrieved API key.

    Raises:
        ValueError: If no key is found.
    """
    api_key = os.getenv(key_name)
    if not api_key:
        raise ValueError(f"Invalid API key: {key_name} not found in environment variables")
    return api_key

def initialize_llm(model_name="meta-llama/llama-3.1-8b-instruct", temperature=0.4, use_streaming=True):
    """
    Initialize a streaming Large Language Model (LLM) with specified parameters.

    Args:
        model_name (str): The model's name.
        temperature (float): Sampling temperature for generation.
        use_streaming (bool): Whether to enable streaming output.

    Returns:
        ChatOpenAI: Instance of the language model.
    """
    api_key = get_api_key()
    callbacks = [StreamingStdOutCallbackHandler()]
    llm = ChatOpenAI(
        model_name=model_name,
        temperature=temperature,
        streaming=use_streaming,
        callbacks=callbacks,
        openai_api_key=api_key,
        openai_api_base="https://openrouter.ai/api/v1"
    )
    return llm

# Initialize the language model for use throughout the notebook
llm = initialize_llm()

python-dotenv could not parse statement starting at line 12


In [3]:
def save_file(data, filename, uniqueness_method="uuid"):
    """
    Save data as a markdown file in a uniquely named manner.

    Args:
        data (str): Content to save.
        filename (str): Base filename.
        uniqueness_method (str): How to make filenames unique (uuid, random_string, hash, counter, datetime).

    Returns:
        str: Path to the saved file.
    """
    folder_name = "Travel Plans"
    os.makedirs(folder_name, exist_ok=True)  # Ensure folder exists

    # Choose unique identifier method for filename
    if uniqueness_method == "uuid":
        unique_id = str(uuid.uuid4())[:8]
    elif uniqueness_method == "random_string":
        chars = string.ascii_letters + string.digits
        unique_id = ''.join(random.choice(chars) for _ in range(8))
    elif uniqueness_method == "hash":
        content_hash = hashlib.md5((data + str(datetime.now())).encode()).hexdigest()[:8]
        unique_id = content_hash
    elif uniqueness_method == "counter":
        counter = 1
        while True:
            test_filename = f"{filename}_{counter}.md"
            test_path = os.path.join(folder_name, test_filename)
            if not os.path.exists(test_path):
                unique_id = str(counter)
                break
            counter += 1
    elif uniqueness_method == "datetime":
        unique_id = datetime.now().strftime("%Y%m%d%H%M%S")
    else:
        unique_id = str(uuid.uuid4())[:8]

    final_filename = f"{filename}_{unique_id}.md"
    file_path = os.path.join(folder_name, final_filename)

    # Write data to file
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(data)
        print(f"File '{file_path}' created successfully.")

    return file_path

def show_md_file(file_path):
    """
    Display the content of a markdown file within the notebook.

    Args:
        file_path (str): Path to the markdown file.
    """
    with open(file_path, "r", encoding = "utf-8") as file:
        content = file.read()
    display(Markdown(content))

# 3. Planner State Structure
Defines a TypedDict, PlannerState, which holds all state for the travel planning workflow:

- travel_preferences: User’s trip preferences.
- destination_options: Suggested destinations.
- selected_destination: User’s chosen destination.
- budget_plan: Trip budget breakdown.
- itinerary: Day-by-day trip plan.
- bookings: Booking recommendations.
- feedback: User feedback.
- final_trip: The final trip package.
messages: Message history (for context).
status: Workflow status string.

In [4]:
class PlannerState(TypedDict):
    """
    Structure representing the entire travel planning state across all workflow nodes.
    """
    travel_preferences: Dict
    destination_options: List[Dict]
    selected_destination: Dict
    budget_plan: Dict
    itinerary: List[Dict]
    bookings: Dict
    feedback: Dict
    final_trip: Dict
    messages: List
    status: str

# 4. Workflow Node Functions
Each node is a function that takes and returns a PlannerState object, incrementally enriching the plan:

- input_collector:
    - Extracts user travel preferences from the message history via the LLM.
- destination_research: 
    - Suggests 3–5 destinations matching the user’s preferences, with descriptions, costs, and timing.
- collect_selected_destination:
    - Captures which destination the user selects from the options.
- budget_planner:
    - Breaks down the trip’s expected costs (transport, accommodation, activities, etc.).
- itinerary_builder:
    - Generates a detailed multi-day plan, balancing activity and rest.
- booking_assistant: 
    - Provides booking recommendations (flights, hotels, activities, insurance, etc.), with tips and links.
- trip_exporter: 
    - Compiles the entire trip—including preferences, itinerary, budget, bookings, packing tips, and emergency contacts—into a final Markdown document.



In [5]:
def input_collector(state: PlannerState) -> PlannerState:
    """
    Collects and organizes user's travel preferences from messages.

    Args:
        state (PlannerState): The current workflow state.

    Returns:
        PlannerState: Updated state with extracted preferences.
    """
    messages = state["messages"]
    # Extract the latest user message
    human_message = next((m for m in reversed(messages) if isinstance(m, HumanMessage)), None)
    system_prompt = (
        "You are a travel input collector. Extract travel preferences from the user's message.\n"
        "Include: destination interests, travel dates, budget range, number of travelers, "
        "accommodation preferences, activity interests, and any special requirements."
    )
    response = llm.invoke([
        SystemMessage(content=system_prompt),
        human_message
    ])
    preferences = {
        "raw_input": human_message.content,
        "processed_preferences": response.content,
        "timestamp": datetime.now().isoformat()
    }
    
    #Update state
    state["travel_preferences"] = preferences
    state["messages"].append(AIMessage(content="I've collected your travel preferences. Moving on to destination research."))
    state["status"] = "preferences_collected"
    return state

In [6]:
def destination_research(state: PlannerState) -> PlannerState:
    """
    Research and suggest possible destinations based on user preferences.

    Args:
        state (PlannerState): The current workflow state.

    Returns:
        PlannerState: Updated state with destination suggestions.
    """
    preferences = state["travel_preferences"]
    system_prompt = """
        You are a destination research expert with extensive global travel knowledge.

        Based on the user's comprehensive travel preferences, suggest 3–5 meticulously selected destinations.

        For each destination, provide:

        1. **Name & Description**:
        - Detailed name
        - Compelling description highlighting unique features

        2. **Preference Alignment**:
        - Explanation of why it matches their specific preferences

        3. **Seasonal Considerations**:
        - Ideal visiting months
        - Weather/climate context

        4. **Cost Analysis**:
        - Luxury, mid-range, and budget cost breakdowns

        5. **Attractions** (categorized):
        - Cultural
        - Natural
        - Adventure
        - Other relevant categories

        6. **Local Transportation**:
        - Public and private options

        7. **Safety Considerations**:
        - Travel advisories
        - Health/safety tips

        8. **Visa Requirements**:
        - Entry rules by nationality (generalized)

        9. **Language Considerations**:
        - Common languages
        - Tips for non-native speakers

        10. **Insider Recommendations**:
            - Hidden gems
            - Local customs
            - Experiences enhancing their travel style
            - Provide links to websites if possible

        Also include **at least one unexpected but highly relevant destination** to broaden their perspective.
        """

    response = llm.invoke([
        SystemMessage(content=system_prompt),
        HumanMessage(content=f"Travel Preferences: {preferences}")
    ])
    destination_options = {
        "options": response.content,
        "research_timestamp": datetime.now().isoformat()
    }
    state["destination_options"] = destination_options
    state["messages"].append(AIMessage(content="I've researched some destinations that match your preferences. Moving on to budget planning."))
    state["status"] = "awaiting_destination_selection"
    return state

In [7]:
def collect_selected_destination(state: PlannerState) -> PlannerState:
    """
    Capture the user's selected destination from the message history.

    Args:
        state (PlannerState): The current workflow state.

    Returns:
        PlannerState: Updated state with the user's destination choice.
    """
    messages = state["messages"]
    latest_human = next((m for m in reversed(messages) if isinstance(m, HumanMessage)), None)
    if not latest_human:
        raise ValueError("No user input found for destination selection.")

    selected_destination = {
        "name": latest_human.content,
        "details": "To be looked up or parsed from previous suggestions."
    }
    state["selected_destination"] = selected_destination
    state["messages"].append(AIMessage(content=f"Thanks! You've selected **{latest_human.content}**. We'll proceed to budget planning."))
    state["status"] = "destination_selected"
    return state

In [8]:
def budget_planner(state: PlannerState) -> PlannerState:
    """
    Create a detailed budget breakdown for the user's trip.

    Args:
        state (PlannerState): The current workflow state.

    Returns:
        PlannerState: Updated state with the trip budget plan.
    """
    preferences = state["travel_preferences"]
    destination = state["selected_destination"]
    system_prompt = """
        You are a travel budget planner with expertise in global financial planning.

        Create an extensively detailed budget breakdown for the trip, including:

        1. **Transportation Costs**
        - International flights
        - Regional connections
        - Airport transfers
        - Daily local transit options

        2. **Accommodation Costs**
        - Based on selected property types
        - Include seasonal price variations

        3. **Daily Expenses**
        - Breakfast, lunch, dinner, snacks
        - Guided tours and self-guided activities
        - Entrance fees
        - Tipping customs by region

        4. **Entertainment Budget**
        - Events, shows, nightlife, excursions

        5. **Shopping Allowances**
        - Souvenirs and discretionary spending

        6. **Communication Expenses**
        - SIM cards, WiFi, data plans

        7. **Recommended Spending Money**
        - Adjusted for destination-specific cost levels

        8. **Extras & Contingencies**
        - Unexpected costs
        - Emergency buffer (with percentage recommendations)

        9. **Currency Exchange Considerations**
        - Local exchange rates
        - Preferred exchange methods

        10. **Money-Saving Strategies**
            - Discounts, passes, bundled offers
            - Budget travel hacks

        11. **Payment Method Recommendations**
            - Cash vs card vs mobile wallets
            - Region-specific guidance

        Be sure to present this budget in both:
        - **Per-person cost**
        - **Total group cost**
        """

    response = llm.invoke([
        SystemMessage(content=system_prompt),
        HumanMessage(content=f"Travel preferences: {preferences}, Selected destination: {destination}")
    ])
    budget_plan = {
        "breakdown": response.content,
        "timestamp": datetime.now().isoformat()
    }
    state["budget_plan"] = budget_plan
    state["messages"].append(AIMessage(content="I've created a budget plan for your trip. Now building your itinerary."))
    state["status"] = "budget_planned"
    return state

In [9]:
def itinerary_builder(state: PlannerState) -> PlannerState:
    """
    Build a detailed day-by-day itinerary for the trip.

    Args:
        state (PlannerState): The current workflow state.

    Returns:
        PlannerState: Updated state with the trip itinerary.
    """
    preferences = state["travel_preferences"]
    destination = state["selected_destination"]
    budget = state["budget_plan"]
    system_prompt = """
        You are a travel itinerary expert specializing in optimized travel experiences.

        Create a meticulously detailed **day-by-day itinerary** for the trip, including:

        1. **Arrival & Departure Logistics**
        - Strategic planning for flights and check-ins
        - Recovery time considerations after travel

        2. **Daily Activities & Attractions**
        - Thoughtfully sequenced to reduce transit time
        - Maximize cultural and experiential value

        3. **Meal Planning**
        - Restaurant recommendations for each meal
        - Include alternative options for variety and flexibility

        4. **Rest & Recovery**
        - Appropriately timed breaks based on activity level

        5. **Special Experiences**
        - Unique highlights strategically placed across the itinerary

        6. **Contingency Planning**
        - Suggestions for weather disruptions or closures

        7. **Timing & Accessibility**
        - Opening hours of all scheduled attractions
        - Walking distances and expected durations

        8. **Transit Planning**
        - Local transportation options with estimated travel times

        9. **Crowd Avoidance Tips**
        - Location-specific strategies to skip long lines

        10. **Photography & Visual Highlights**
            - Recommended photo opportunities by location and time of day

        11. **Cultural Etiquette Reminders**
            - Important do's and don'ts for each region or activity

        Ensure the itinerary:
        - Is **balanced** to avoid exhaustion
        - **Maximizes authenticity and enjoyment**
        - Respects **budget constraints** and **traveler preferences**
        """
    response = llm.invoke([
        SystemMessage(content=system_prompt),
        HumanMessage(content=f"Travel preferences: {preferences}, Selected destination: {destination}, Budget plan: {budget}")
    ])
    itinerary = {
        "daily_plan": response.content,
        "timestamp": datetime.now().isoformat()
    }
    state["itinerary"] = itinerary
    state["messages"].append(AIMessage(content="Your itinerary is ready! Moving on to booking assistance."))
    state["status"] = "itinerary_built"
    return state

In [10]:
def booking_assistant(state: PlannerState) -> PlannerState:
    """
    Provide booking recommendations and strategies for the trip.

    Args:
        state (PlannerState): The current workflow state.

    Returns:
        PlannerState: Updated state with booking information.
    """
    preferences = state["travel_preferences"]
    destination = state["selected_destination"]
    itinerary = state["itinerary"]
    system_prompt = """
        You are a travel booking assistant with expertise in reservation strategies.

        Provide **extensively detailed booking recommendations**, including:

        1. **Flight Options**
        - Multiple routes with specific airlines
        - Fare comparison strategies
        - Optimal booking times
        - Layover considerations

        2. **Accommodation Recommendations**
        - Curated listings with platform/direct booking links
        - Property amenities (WiFi, breakfast, accessibility, etc.)
        - Neighborhood advantages and safety

        3. **Activity Reservations**
        - Required bookings (e.g., tours, attractions, events)
        - Advance booking timelines
        - Skip-the-line or VIP access options

        4. **Transportation Bookings**
        - Car rentals (providers, pickup/drop-off info)
        - Train passes or city transport cards
        - Local transit options with pre-booking tips

        5. **Travel Insurance**
        - Plan recommendations by destination/activities
        - Medical, trip cancellation, and baggage coverage
        - Insurance provider comparisons
        - Suggest various insurance options with links to websites

        6. **Baggage Considerations**
        - Airline-specific policies (carry-on vs. checked)
        - Packing strategies and weight tips

        7. **Loyalty Programs & Upgrades**
        - Opportunities to earn/redeem miles or points
        - Upgrade strategies (timing, check-in tips)

        8. **Cancellation Policies**
        - Refund eligibility and cancellation windows
        - Suggested flexible booking options

        9. **Booking Confirmations**
        - Best practices for storing confirmations
        - Centralized digital folders or apps

        10. **Seat Selection Tips**
            - Ideal locations by plane type
            - Avoiding middle seats, maximizing comfort

        11. **Seasonal Pricing Trends**
            - High/low season comparisons
            - Booking windows by destination and event

        12. **Expert Tips**
            - Price tracking tools and alerts
            - Aggregator sites vs. direct booking trade-offs
            - Timeline checklist to ensure timely reservations

        Ensure all recommendations are **optimized for availability, comfort, and cost-effectiveness**.
        """

    response = llm.invoke([
        SystemMessage(content=system_prompt),
        HumanMessage(content=f"Travel preferences: {preferences}, Selected destination: {destination}, Itinerary: {itinerary}")
    ])
    bookings = {
        "recommendations": response.content,
        "timestamp": datetime.now().isoformat()
    }
    state["bookings"] = bookings
    state["messages"].append(AIMessage(content="Here are your booking recommendations. Please provide feedback on the plan."))
    state["status"] = "bookings_provided"
    return state

In [11]:
def trip_exporter(state: PlannerState) -> PlannerState:
    """
    Compile all trip details into a final exportable document.

    Args:
        state (PlannerState): The current workflow state.

    Returns:
        PlannerState: Updated state containing the final trip package.
    """
    preferences = state["travel_preferences"]
    destination = state["selected_destination"]
    budget = state["budget_plan"]
    itinerary = state["itinerary"]
    bookings = state["bookings"]
    system_prompt = """
        You are a travel document preparation specialist with exceptional organizational skills.

        Create a **comprehensive and meticulously structured trip package** that includes:

        1. **Trip Summary**
        - All key logistical elements
        - Contact information for accommodations, transport, and activities
        - Include website links whenever possible

        2. **Chronological Itinerary**
        - Complete day-by-day schedule with precise timings
        - Transportation details and location coordinates (e.g., maps or GPS-ready links)

        3. **Budget Breakdown**
        - Itemized costs by category (e.g., food, lodging, transport, entertainment)
        - Include simple tracking mechanisms (tables, checklists, or trackers)

        4. **Booking & Confirmation Details**
        - Organized by category: flights, hotels, tours, activities
        - Include reference numbers and contact info

        5. **Packing Recommendations**
        - Tailored to destination's climate and activities
        - Include weather-specific gear and region-specific items

        6. **Clothing Suggestions**
        - Outfit suggestions per day, activity, or occasion

        7. **Essential Documents Checklist**
        - Passports, IDs, visas, insurance, tickets, etc.

        8. **Medication Considerations**
        - Prescriptions, travel sickness, vaccination records

        9. **Electronic Device Prep**
        - Adapter types, power banks, connectivity tools

        10. **Cultural Etiquette Guidelines**
            - Dos and don'ts for the destination country or region

        11. **Language Essentials**
            - Key phrases for greetings, transportation, dining, and emergencies

        12. **Safety Information**
            - Travel advisories, personal safety tips

        13. **Emergency Resources**
            - Local emergency numbers
            - Nearest embassies/consulates
            - Closest hospitals or medical centers
            - Provide the website links and or phone numbers of the embassies, hospitals or medical centres/consulates

        14. **Organization Tips**
            - Digital folder structure (e.g., PDFs, e-tickets)
            - Physical folder layout (e.g., binders, wallets)

        15. **Return Preparation Guidance**
            - Checklist for homecoming (e.g., airport transit, customs, unpacking priorities)

        Deliver this as a complete **travel document system**, formatted for:
        - **Printer-friendly use**
        - **Mobile-optimized access**

        Ensure it is easy to reference throughout the journey.
        """

    response = llm.invoke([
        SystemMessage(content=system_prompt),
        HumanMessage(content=f"""
        Travel preferences: {preferences}
        Selected destination: {destination}
        Budget plan: {budget}
        Itinerary: {itinerary}
        Booking details: {bookings}
        """)
    ])
    final_trip = {
        "package": response.content,
        "created_at": datetime.now().isoformat()
    }
    state["final_trip"] = final_trip
    state["messages"].append(AIMessage(content="Your trip package is ready! Here's everything you need for your journey."))
    state["status"] = "trip_exported"
    return state

# 5. Workflow Construction with LangGraph

- Adds each function as a node to a StateGraph.
- Specifies node execution order (edges).
- Entry point: input_collector.
- Terminal node: trip_exporter (signals workflow completion).
- Compiles the workflow into a graph object.

In [12]:
from langgraph.graph import StateGraph, END

# Create the workflow graph structure
workflow = StateGraph(PlannerState)

# Add all workflow nodes
workflow.add_node("input_collector", input_collector)
workflow.add_node("destination_research", destination_research)
workflow.add_node("collect_selected_destination", collect_selected_destination)
workflow.add_node("budget_planner", budget_planner)
workflow.add_node("itinerary_builder", itinerary_builder)
workflow.add_node("booking_assistant", booking_assistant)
workflow.add_node("trip_exporter", trip_exporter)

# Define workflow logic (edges)
workflow.set_entry_point("input_collector")
workflow.add_edge("input_collector", "destination_research")
workflow.add_edge("destination_research", "collect_selected_destination")
workflow.add_edge("collect_selected_destination", "budget_planner")
workflow.add_edge("budget_planner", "itinerary_builder")
workflow.add_edge("itinerary_builder", "booking_assistant")
workflow.add_edge("booking_assistant", "trip_exporter")
workflow.add_edge("trip_exporter", END)

# Compile workflow for execution
graph = workflow.compile()

# 6. Main Interaction Loop
- process_travel_query(query, planner_state):
    - Handles one user query, advancing the workflow, updating state, and returning assistant responses (or errors).
    - Accepts "exit" to terminate.
    - "continue" advances to next step.
- Sample Interaction:
    - Simulates a conversation, starting with a user’s preferences, continuing through destination selection, and ending with trip export and Markdown file creation.

In [13]:
def process_travel_query(query, planner_state):
    """
    Process a single user query, advancing the workflow and updating state.

    Args:
        query (str): The user's input query.
        planner_state (PlannerState): The current state of the planner.

    Returns:
        tuple: (response (str), updated_state (PlannerState))
    """
    if query.lower() == "exit":
        return "Exiting the travel planner.", planner_state

    # Add new user input to the state, unless just continuing
    if query.lower() != "continue":
        planner_state["messages"].append(HumanMessage(content=query))


    try:
        assistant_response = ""
        for output in graph.stream(planner_state):
            for node, updated_state in output.items():
                print(f"\n[Node: {node}]")
                # Collect new assistant messages
                for message in updated_state["messages"]:
                    if isinstance(message, AIMessage) and message not in planner_state["messages"]:
                        assistant_response += f"{message.content}\n"
                planner_state = updated_state

                # If workflow is complete, export the trip package
                if planner_state["status"] == "trip_exported":
                    final_package = planner_state["final_trip"].get("package", "No package available.")
                    assistant_response += f"\nFinal Trip Package:\n{final_package}\n\nPlanning complete!"
                    return assistant_response, planner_state
        # Prompt user based on workflow state
        if not assistant_response:
            if planner_state["status"] == "initial":
                assistant_response = "Enter your travel preferences (e.g., destination, dates, budget, etc.): "
            elif planner_state["status"] == "awaiting_destination_selection":
                options = planner_state.get("destination_options", {}).get("options", "No options available yet.")
                assistant_response = f"\nSuggested destinations:\n{options}\nPlease select a destination from the suggested options: "
            else:
                assistant_response = "Provide any additional input or feedback (or type 'continue' to proceed): "

        return assistant_response, planner_state

    except Exception as e:
        error_message = (
            f"Error occurred: {e}\nStack trace:\n{traceback.format_exc()}\n"
            "Please check your setup (e.g., API keys, dependencies) and try again."
        )
        return error_message, planner_state

In [ ]:
# Initial empty state for the workflow
state = PlannerState(
    travel_preferences={},
    destination_options=[],
    selected_destination={},
    budget_plan={},
    itinerary=[],
    bookings={},
    feedback={},
    final_trip={},
    messages=[],
    status="initial"
)

#simulate several queries
queries = [
    """
I'm planning a family trip to Kenya in June 2025 with my spouse and two children (ages 10 and 14).

- **Budget:** $8,000 (excluding international flights)  
- **Activity Interests:**  
  - Water activities including snorkeling, swimming, and possibly kayaking or sailing along the coast  
  - Wildlife safaris focusing on game drives and guided wildlife viewing in national parks such as Maasai Mara and Amboseli  
- **Accommodation Preferences:** Mid-range hotels or lodges with family rooms or interconnected rooms that offer comfort and kid-friendly amenities  
- **Trip Duration:** 10-12 days  
- **Special Requirements:** No dietary restrictions or mobility concerns; standard health precautions apply

    """,
    "continue",
    "Nairobi sounds good, let's select it.",
    "exit"
]


print("Welcome to the Travel Planning Assistant!")
current_state = state  # Use the initialized state

for query in queries:
    response, current_state = process_travel_query(query, current_state)
    print(f"\nAssistant: {response}")

    # Save and show response if workflow is complete
    if query.lower() == "exit" or current_state["status"] == "trip_exported":
        file_path = save_file(response, filename="travel_plan", uniqueness_method="datetime")
        clear_output()
        show_md_file(file_path)
        break


Final Trip Package:
**Comprehensive Travel Document System for Kenya Family Trip (June 2025)**

**Trip Summary**

* Destination: Kenya
* Travel Dates: June 2025
* Budget: $8,000 (excluding international flights)
* Number of Travelers: 4 (you, your spouse, and two children aged 10 and 14)
* Accommodation Preferences: Mid-range hotels or lodges with family rooms or interconnected rooms that offer comfort and kid-friendly amenities
* Activity Interests: Water activities (snorkeling, swimming, kayaking, sailing), Wildlife safaris (game drives, guided wildlife viewing in national parks)
* Special Requirements: No dietary restrictions, no mobility concerns, standard health precautions apply

**Contact Information**

* Accommodations:
	+ Hotel Boulevard (Nairobi): +254 20 222 3333, [www.hotelboulevard.com](http://www.hotelboulevard.com)
	+ Mara Serena Lodge (Maasai Mara National Reserve): +254 20 222 4444, [www.maraseralodge.com](http://www.maraseralodge.com)
	+ Amboseli Serena Lodge (Amboseli National Park): +254 20 222 5555, [www.amboseliserenadolodge.com](http://www.amboseliserenadolodge.com)
	+ Hotel English Point (Mombasa): +254 20 222 6666, [www.hotelenglishpoint.com](http://www.hotelenglishpoint.com)
* Transportation:
	+ Jomo Kenyatta International Airport (NBO): +254 20 222 7777, [www.jkia.go.ke](http://www.jkia.go.ke)
	+ Kenya Airways: +254 20 222 8888, [www.kenya-airways.com](http://www.kenya-airways.com)
* Activities:
	+ Abercrombie & Kent: +254 20 222 9999, [www.abercrombiekent.com](http://www.abercrombiekent.com)
	+ African Wildlife Foundation: +254 20 222 1111, [www.awf.org](http://www.awf.org)

**Chronological Itinerary**

* **Day 1: Arrival in Nairobi**
	+ Arrive at Jomo Kenyatta International Airport (NBO)
	+ Transfer to Hotel Boulevard (approximately 1 hour)
	+ Check-in at Hotel Boulevard
	+ Spend the day relaxing and acclimating to the local environment
	+ Dinner at Carnivore Restaurant
* **Day 2: Nairobi to Maasai Mara National Reserve**
	+ Early morning departure from Nairobi to Maasai Mara National Reserve (approximately 5 hours)
	+ Check-in at Mara Serena Lodge
	+ Afternoon game drive in the reserve
	+ Dinner at Mara Serena Lodge
* **Day 3: Maasai Mara National Reserve**
	+ Morning game drive in the reserve
	+ Afternoon game drive in the reserve
	+ Optional: hot air balloon ride over the reserve (approximately $500 per person)
	+ Dinner at Mara Serena Lodge
* **Day 4: Maasai Mara National Reserve**
	+ Full-day game drive in the reserve
	+ Optional: visit to a local Maasai village (approximately $20 per person)
	+ Dinner at Mara Serena Lodge
* **Day 5: Maasai Mara National Reserve to Amboseli National Park**
	+ Early morning departure from Maasai Mara National Reserve to Amboseli National Park (approximately 4 hours)
	+ Check-in at Amboseli Serena Lodge
	+ Afternoon game drive in the park
	+ Dinner at Amboseli Serena Lodge
* **Day 6: Amboseli National Park**
	+ Morning game drive in the park
	+ Afternoon game drive in the park
	+ Optional: visit to a local elephant sanctuary (approximately $30 per person)
	+ Dinner at Amboseli Serena Lodge
* **Day 7: Amboseli National Park**
	+ Full-day game drive in the park
	+ Optional: visit to a local Maasai village (approximately $20 per person)
	+ Dinner at Amboseli Serena Lodge
* **Day 8: Amboseli National Park to Mombasa**
	+ Early morning departure from Amboseli National Park to Mombasa (approximately 2 hours)
	+ Check-in at Hotel English Point
	+ Spend the day relaxing and enjoying the beach
	+ Dinner at Nyali Beach Restaurant
* **Day 9: Mombasa**
	+ Morning snorkeling or swimming excursion (approximately $50 per person)
	+ Afternoon sailing or kayaking excursion (approximately $50 per person)
	+ Dinner at Mama Mia Restaurant
* **Day 10: Mombasa**
	+ Full-day beach relaxation and water activities
	+ Optional: visit to a local marine park (approximately $30 per person)
	+ Dinner at Tamarind Restaurant
* **Day 11: Mombasa**
	+ Morning visit to a local market (approximately $20 per person)
	+ Afternoon visit to a local cultural center (approximately $20 per person)
	+ Dinner at Karibu Restaurant
* **Day 12: Departure from Mombasa**
	+ Spend the morning relaxing at the hotel or engaging in last-minute shopping
	+ Depart for Jomo Kenyatta International Airport (NBO) for international flights

**Budget Breakdown**

* Transportation: $12,400 - $18,800
* Accommodation: $15,000 - $36,000
* Daily expenses: $12,800 - $27,200
* Entertainment budget: $6,000 - $13,600
* Shopping allowance: $800 - $2,000
* Communication expense: $800 - $2,000
* Recommended spending money: $2,000 - $4,000
* Extras & contingencies: $1,200 - $2,400
* Money-saving strategies: $1,920 - $3,840

**Booking & Confirmation Details**

* Flights:
	+ International flights: booked with Kenya Airways or Emirates
	+ Domestic flights: booked with Kenya Airways or Fly540
* Accommodations:
	+ Hotel Boulevard (Nairobi): booked through Booking.com or Expedia
	+ Mara Serena Lodge (Maasai Mara National Reserve): booked through Booking.com or Hotels.com
	+ Amboseli Serena Lodge (Amboseli National Park): booked through Booking.com or Hotels.com
	+ Hotel English Point (Mombasa): booked through Booking.com or Expedia
* Activities:
	+ Abercrombie & Kent: booked through their website or a travel agent
	+ African Wildlife Foundation: booked through their website or a travel agent

**Packing Recommendations**

* Weather-specific gear:
	+ Lightweight clothing for warm weather
	+ Rain gear (umbrella, raincoat)
	+ Sun protection (sunscreen, sunglasses)
* Region-specific items:
	+ Insect repellent
	+ Power adapter for Kenya (Type G)
	+ Water bottle or refillable container

**Clothing Suggestions**

* Outfit suggestions per day:
	+ Day 1: casual clothing for arrival and relaxation
	+ Day 2-3: comfortable clothing for game drives and outdoor activities
	+ Day 4-5: dressier clothing for dinner at Mara Serena Lodge
	+ Day 6-7: comfortable clothing for game drives and outdoor activities
	+ Day 8-9: beachwear for relaxation and water activities
	+ Day 10-11: casual clothing for local market and cultural center visits
	+ Day 12: comfortable clothing for departure

**Essential Documents Checklist**

* Passports
* IDs
* Visas (if required)
* Travel insurance documents
* Flight tickets
* Accommodation confirmations
* Activity bookings
* Travel itinerary

**Medication Considerations**

* Prescriptions:
	+ Any prescription medications should be packed in the original packaging with a copy of the prescription
* Travel sickness:
	+ Consider packing motion sickness medication or wristbands
* Vaccination records:
	+ Ensure all travelers have up-to-date vaccination records

**Electronic Device Prep**

* Adapters:
	+ Kenya uses Type G power sockets, which are the same as those in the UK
* Power banks:
	+ Consider packing portable power banks for charging devices on the go
* Connectivity tools:
	+ Bring a portable Wi-Fi hotspot or rent one locally

**Cultural Etiquette Guidelines**

* Respect local customs and traditions
* Dress modestly when visiting local villages or attending cultural events
* Remove shoes when entering local homes or places of worship
* Use local greetings and phrases to show respect and appreciation

**Language Essentials**

* Key phrases for greetings:
	+ Hello: Jambo (HAHM-boh)
	+ Goodbye: Habari (HAH-bah-ree)
* Key phrases for transportation:
	+ Where is...?: Eneo gani...? (EH-neh-oh GAH-nee)
	+ How much?: Gani? (GAH-nee)
* Key phrases for dining:
	+ I'd like...: Ninataka... (NEE-nah-tah-kah)
	+ The bill, please: Fomula, tafadhali (FOH-moo-lah TAH-fah-dah-lee)
* Key phrases for emergencies:
	+ Help!: Msaada! (MEE-sah-dah)
	+ Police!: Polisi! (POH-lee-see)

**Safety Information**

* Travel advisories:
	+ Check the official government website for any travel advisories or warnings
* Personal safety tips:
	+ Be aware of your surroundings and keep valuables secure
	+ Avoid traveling alone at night or in unfamiliar areas

**Emergency Resources**

* Local emergency numbers:
	+ Police: 999
	+ Ambulance: 999
	+ Fire department: 999
* Nearest embassies/consulates:
	+ US Embassy: [www.ustravel.state.gov](http://www.ustravel.state.gov)
	+ Canadian Embassy: [www.travel.gc.ca](http://www.travel.gc.ca)
* Closest hospitals or medical centers:
	+ Nairobi Hospital: +254 20 222 3333
	+ Mombasa Hospital: +254 20 222 4444

**Organization Tips**

* Digital folder structure:
	+ Create a folder for each day of the trip with relevant documents and confirmations
	+ Use a cloud storage service like Google Drive or Dropbox to access documents from anywhere
* Physical folder layout:
	+ Use a binder or folder to keep all documents and confirmations organized
	+ Consider using a travel wallet or pouch to keep important documents secure

**Return Preparation Guidance**

* Checklist for homecoming:
	+ Ensure all documents and confirmations are in order
	+ Pack any necessary items for the return journey
	+ Confirm flight and accommodation details for the return journey

Planning complete!